In [ ]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass  # To get the password without showing the input
password = getpass.getpass()

In [ ]:
connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila'
engine = create_engine(connection_string)

In [ ]:
# 1. Create a query or queries to extract the information you think may be relevant for building the prediction model. It should include some film features and some rental features (X).
query = '''
    SELECT f.film_id, f.title, f.release_year, f.length, f.rating, f.special_features,
           r.rental_id, r.rental_date, p.amount AS rental_cost
    FROM film f
    INNER JOIN inventory i ON f.film_id = i.film_id
    INNER JOIN rental r ON i.inventory_id = r.inventory_id
    INNER JOIN payment p ON r.rental_id = p.rental_id
    WHERE r.rental_date >= '2005-05-01' AND r.rental_date <= '2005-05-31'
'''

# Read the data into a pandas dataframe
data = pd.read_sql_query(query, engine)
data.head()

In [ ]:
data.shape

In [ ]:
# 2. Create a query to get the list of all unique film titles and a boolean indicating if it was rented (rental_date) in May 2005. (Create new column called - 'rented_in_may'). This will be our TARGET (y) variable.



query = ''' SELECT film_id, title as movie_title, category_id, length as movie_length, rating, rental_duration, IF(rental_date BETWEEN '2005-05-01' AND '2005-05-31', 1, 0) AS rented_in_may,
       (SELECT COUNT(*) FROM rental 
        WHERE inventory_id IN (SELECT inventory_id FROM inventory WHERE film_id = film.film_id) 
          AND rental_date BETWEEN '2005-05-01' AND '2005-05-31') AS rentals_in_may
        FROM film
        LEFT JOIN inventory USING(film_id)
        LEFT JOIN film_category USING(film_id)
        LEFT JOIN category USING(category_id)
        LEFT JOIN rental USING(inventory_id)
        GROUP BY film_id;'''




In [ ]:
#3 Read the data into a Pandas dataframe. At this point you should have 1000 rows. Number of columns depends on the number of features you chose.

data = pd.read_sql_query(query, engine)
data.head()

In [ ]:
# Analyze extracted features (X) and transform them. You may need to encode some categorical variables, or scale numerical variables.

In [ ]:
X_num = data.select_dtypes(include='number')
X_cat = data.select_dtypes(include='object')
y = data['rented_in_may']

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_cat = pd.get_dummies(X_cat['rating'])
X_cat

minMax = MinMaxScaler()
minMax.fit(X_num)
X_minmax_scaled = minMax.transform(X_num)
X_minmax_scaled = pd.DataFrame(X_minmax_scaled,columns=X_num.columns)
X_minmax_scaled

X = pd.concat([X_num, X_cat], axis=1)
X

correlations_matrix = X.corr()
sns.heatmap(correlations_matrix, annot=True)
plt.show()

X.drop(['release_year'], axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

transformer = StandardScaler()
transformer.fit(X_train)

X_train_scaled_np = transformer.transform(X_train)
X_train_scaled = pd.DataFrame(X_train_scaled_np, columns=X_train.columns, index=X_train.index) 
X_test_scaled_np = transformer.transform(X_test)
X_test_scaled = pd.DataFrame(X_test_scaled_np , columns=X_test.columns, index=X_test.index)
X_train_scaled.head()

In [ ]:
# Create a logistic regression model to predict 'rented_in_may' from the cleaned data.

In [ ]:
y_test_pred = logreg1.predict(X_test_transformed)

In [ ]:
# Evaluate the results.

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

cm_test = confusion_matrix(y_test,y_test_pred) 
disp = ConfusionMatrixDisplay(confusion_matrix=cm_test, display_labels=logreg1.classes_) 
disp.plot()
plt.show()